<a href="https://colab.research.google.com/github/jangvu/Project_by_me/blob/main/Steel_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/Data"

In [ ]:
!unzip -q "/content/drive/My Drive/Data/NEU-DET.zip"


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [ ]:
import csv 
#import requests 
import xml.etree.ElementTree as ET 

# **Loading data**

In [ ]:
IMAGES_PATH = '/content/IMAGES'
ANNOTATIONS_PATH = '/content/ANNOTATIONS'

IMAGES_DIR = os.listdir(IMAGES_PATH)
ANNOTATIONS_DIR = os.listdir(ANNOTATIONS_PATH)

images_arr = []
annotations_arr = []
for image in IMAGES_DIR:
    images_arr.append(IMAGES_PATH + "/" + image)
for annotation in ANNOTATIONS_DIR:
    annotations_arr.append(ANNOTATIONS_PATH + "/" + annotation)

images_arr.sort()
annotations_arr.sort()

## **Creating an ImageOject to save data**

In [ ]:
class imageObject:
  def __init__(self,_filePath, _name, _label, _xmin, _ymin, _xmax, _ymax, _width = 200, _height = 200):
    self.name = _name
    self.label = _label
    self.width = _width
    self.height = _height
    self.xmin = _xmin
    self.ymin = _ymin
    self.xmax = _xmax
    self.ymax = _ymax
    self.filePath = _filePath



## **Resize Img and Bounding box**

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
data_image = []
labels_image = []
boxes_image = []
images_path = []
new_size = 224
size = 224
boxed_image_arr = []


#  data_image.append(img_to_array(image))

#Bounding box Resize

for i in range(len(annotations_arr)):
    xmlFile = open(annotations_arr[i])
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    filePath = images_arr[i]
    name = root.find('filename').text
    label = name.split("_")[0]

    for boundingObject in root.findall('object'):
        xmin = float(boundingObject.find('bndbox').find('xmin').text)/200
        ymin = float(boundingObject.find('bndbox').find('ymin').text)/200
        xmax = float(boundingObject.find('bndbox').find('xmax').text)/200
        ymax = float(boundingObject.find('bndbox').find('ymax').text)/200
        boxed_image_arr.append(imageObject(filePath, name, label, xmin, ymin, xmax, ymax))

#Image Resize
for img in boxed_image_arr:
  image = load_img(img.filePath, target_size=(size,size))
  data_image.append(img_to_array(image))
  labels_image.append(img.label)
  boxes_image.append([img.xmin, img.ymin, img.xmax, img.ymax])


In [ ]:
print(labels_image[0])
print(boxes_image[0])
len(boxes_image)
#x1,y1,x2,y2 = lambda x: x*200 for x in boxes_image[0]

## **Normalize Image and OneHotCoder**

In [ ]:
data_image = np.array(data_image, dtype="float32") / 255.0
labels_image = np.array(labels_image)
boxes_image = np.array(boxes_image, dtype = "float32") 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# perform one-hot encoding on the labels
labelBinarizer = LabelBinarizer()
labels_image = labelBinarizer.fit_transform(labels_image)

In [ ]:
img_train, img_test, label_train, label_test, box_train, box_test = train_test_split(data_image,labels_image,boxes_image, test_size = 0.2, random_state=42)

In [ ]:
print(len(img_train))
print(len(label_train))
len(img_train[0])

# **Building Model**

In [ ]:
LEARNING_RATE = 1e-4
NUM_EPOCHS = 100
BATCH_SIZE = 32

# load the VGG16 network, ensuring the head FC layers are left off
IMG_SHAPE = (size, size, 3)
vgg=tf.keras.applications.VGG19(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid", name="bounding_box")(bboxHead)
# construct the model we will fine-tune for bounding box regression
# construct a second fully-connected layer head, this one to predict the class label
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(len(labelBinarizer.classes_), activation="softmax", name="class_label")(softmaxHead)
# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model_vgg16 = Model(inputs = vgg.input, outputs = (bboxHead, softmaxHead))

losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error",
}
# define a dictionary that specifies the weights per loss (both the
# class label and bounding box outputs will receive equal weight)
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

# model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# initialize the optimizer, compile the model, and show the model
# summary

def updateLearningRate(epoch, lr):
    print("epoch: {}\tlearning rate: {}".format(epoch, lr))
    if epoch < 10 :
        lr = 0.01
    elif epoch >= 10 and epoch < 50:
        lr = 0.001
    elif epoch >= 50:
        lr = 0.0001
    return lr

early_stopping = keras.callbacks.EarlyStopping(patience=5)
opt = Adam(lr = LEARNING_RATE, decay = LEARNING_RATE / NUM_EPOCHS)


model_vgg16.compile(loss = losses, optimizer = opt, metrics = ["accuracy"], loss_weights = lossWeights)
model_vgg16.summary()

In [ ]:
trainTargets = {
    "class_label": label_train,
    "bounding_box": box_train
}
# construct a second dictionary, this one for our target testing
# outputs
testTargets = {
    "class_label": label_test,
    "bounding_box": box_test
}
history = model_vgg16.fit(img_train,trainTargets,epochs = 100, validation_split=0.1)

In [ ]:
model_vgg16.evaluate(img_test,testTargets)

In [ ]:
losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error",
}
baseModel_res = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False,
                                              input_tensor = Input(shape = (224, 224, 3)))

flatten_res = baseModel_res.output
flatten_res = Flatten()(flatten_res)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead_res = Dense(128, activation="relu")(flatten_res)
bboxHead_res = Dense(64, activation="relu")(bboxHead_res)
bboxHead_res = Dense(32, activation="relu")(bboxHead_res)
bboxHead_res = Dense(4, activation="sigmoid", name="bounding_box")(bboxHead_res)
# construct the model we will fine-tune for bounding box regression
# construct a second fully-connected layer head, this one to predict
# the class label
headModel_res = keras.layers.AveragePooling2D(pool_size = (7, 7))(baseModel_res.output)
headModel_res = Flatten()(headModel_res)
softmaxHead_res = Dense(512, activation="relu")(headModel_res)
softmaxHead_res = Dropout(0.5)(softmaxHead_res)
softmaxHead_res = Dense(512, activation="relu")(softmaxHead_res)
softmaxHead_res = Dropout(0.5)(softmaxHead_res)
softmaxHead_res = Dense(len(labelBinarizer.classes_), activation="softmax", name="class_label")(softmaxHead_res)
# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model_resnet152 = Model(inputs = baseModel_res.input, outputs = (bboxHead_res, softmaxHead_res))

for layer in baseModel_res.layers:
    layer.trainable = False
    
model_resnet152.compile(loss = losses, optimizer = opt, metrics = ['accuracy'])
model_resnet152.summary()

In [ ]:
history_resnet = model_resnet152.fit(img_train,trainTargets, callbacks = [early_stopping],epochs = 200, validation_split=0.2)

In [ ]:
pic1 = img_train[0]*255
box1 = boxes_image[0]
x1 = int(box1[0] * 200)
y1 = int(box1[1] * 200)
x2 = int(box1[2] * 200)
y2 = int(box1[3] * 200)
print("{}-{}-{}-{}".format(x1, y1, x2, y2))
pic1 = pic1.astype(np.uint8)
from PIL import Image
import cv2

pic1 = Image.fromarray(pic1, 'RGB')
pic1.save('pic1.jpg')
image = cv2.imread(IMAGES_PATH + 'scratches_244.jpg')
cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
plt.imshow(image)
plt.show()

# **Plot image and box**

In [ ]:
import cv2

In [ ]:
images_path1 = IMAGES_PATH + "/scratches_244.jpg"
annotation_path1 = ANNOTATIONS_PATH + "/scratches_244.xml"
boxed_arr = []
xmlFile = open(annotation_path1)
tree = ET.parse(xmlFile)
root = tree.getroot()
filename = images_path1
label = root.find('filename').text.split("_")[0]
for boundingObject in root.findall('object'):
    xmin = boundingObject.find('bndbox').find('xmin').text
    ymin = boundingObject.find('bndbox').find('ymin').text
    xmax = boundingObject.find('bndbox').find('xmax').text
    ymax = boundingObject.find('bndbox').find('ymax').text
    boxed_arr.append(imageObject(images_path1, filename, label, xmin, ymin, xmax, ymax))
        
for imageObj in boxed_arr:
    print(imageObj.name)
    print(imageObj.label)
    image = cv2.imread(imageObj.name)
    image = cv2.imread(imageObj.filePath)
#    image = imutils.resize(image, width=224)
    (h, w) = image.shape[:2]
    print(h)
    print(w)
    w1 = 200
    w2 = 224
#     image = cv2.resize(image, (w2, w2))
    print(image.shape)
#     (h, w) = image.shape[:2]
    xmin = int(int(imageObj.xmin) * (w/w1))
    ymin = int(int(imageObj.ymin) * (h/w1))
    xmax = int(int(imageObj.xmax) * (w/w1))
    ymax = int(int(imageObj.ymax) * (h/w1))
    y = ymin - 10 if ymin - 10 > 10 else ymin + 10
    # put text in the picture
    cv2.putText(image, imageObj.label, (xmin, y), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)
    # plot image and box
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    plt.axis("off")
    plt.imshow(image)
    plt.show()